# Praktische vragen Deep Learning

Sabine De Vreese, Stijn Lievens en Simon De Gheselle

29 september 2025

## 1  Practice with `tf.data.Dataset`
### 1.1  Commonly used operations
In this exercise we will practice some commonly used operations oftf.data.Datasetobjects.We start with a (small) Dataset object where each item contains 4 elements.

In [ ]:
import tensorflow as tf
import keras as ks
# Create list of tuples to beconvertedintoDatasetobject
data = [(i, i+1, i+2, 100 + i)for i in range(20)]
ds = tf.data.Dataset.from_tensor_slices(data)
for item in ds.take(2):
    print(item)
# Output:
# tf.Tensor((0, 1, 2, 100), shape=(4,), dtype=int32)
# tf.Tensor((1, 2, 3, 101), shape=(4,), dtype=int32)

1. Create a Dataset where each item is a `tuple(X, y)`, where y is the last element of each item in ds.
yields:
```python
(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>,<tf.Tensor: shape=(), dtype=int32, numpy=100>)(<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>,<tf.Tensor: shape=(), dtype=int32, numpy=101>)
```

In [ ]:
ds1 = ds.map(lambda e: (e[:3],e[3]))
# ds1 = ds.map(lambda e: (e[:-1],e[-1]))

for item in ds1.take(2):
    print(item)

2.Starting from ds only keep those items where the sum of the first threeelements is a multiple of 4. Use methods from `keras.ops` to achieve this.

yields
```python
tf.Tensor([  3   4   5 103], shape=(4,), dtype=int32)tf.Tensor([  7   8   9 107], shape=(4,), dtype=int32)tf.Tensor([ 11  12  13 111], shape=(4,), dtype=int32)tf.Tensor([ 15  16  17 115], shape=(4,), dtype=int32)tf.Tensor([ 19  20  21 119], shape=(4,), dtype=int32
```

In [ ]:
ds2 = ds.filter(lambda e: ks.ops.mod(
    ks.ops.sum(e[:3]),4) == 0)
for item in ds2:
    print(item)

3.  -   (a) Write a method `normalise(x)` that takes a batch of elements x as input where x has shape(batch_size, num_features). The return value is such that each feature (i.e. each column of x) has mean zero and standard deviation equal to one. Use methods from keras.ops.

    -   (b)Transform the original datasetdsby taking the following steps:
        -   Cast the tensors totf.float 32 tensors.
        -   Shuffle the dataset. Make sure that the buffer size is large enoughto shuffle the complete dataset, and use seed=42 for reproducibility.
        -   Create batches of exactly 8 elements each.
        -   Normalise each batch using the method normalise you just wrote

yields:

```python
tf.Tensor([[-0.47733918 -0.47733918 -0.47733918 -0.47733918][ 1.2028948   1.2028948   1.2028948   1.2028948 ]..... (some rows removed)[ 0.2864035   0.2864035   0.2864035   0.2864035 ][-0.9355848  -0.9355848  -0.9355848  -0.9355848 ]], shape=(8, 4), dtype=float32)tf.Tensor([[-0.25607374 -0.25607374 -0.25607374 -0.25607374][ 1.5364425   1.5364425   1.5364425   1.5364425 ]..... (some rows removed)[-0.5121475  -0.5121475  -0.5121475  -0.5121475 ][ 1.2803687   1.2803687   1.2803687   1.2803687 ]], shape=(8, 4), dtype=float32)
```


In [ ]:
def normalise(x):
    mu = ks.ops.mean(x,axis=0)
    sigma = ks.ops.std(x,axis=0)
    return (x-mu)/sigma

ds3 = ds.map(lambda e: tf.cast(e,tf.float32)).shuffle(ds.cardinality(),seed=42).batch(8).map(lambda b : normalise(b))
for item in ds3:
    print(item)

Start with a Dataset consisting of [1,2,...,10]. Apply `create_ds` to each element of this Dataset. You get a Dataset where each item itself is a Dataset! Make sure you understand the output of the following code

In [ ]:

def create_ds(i):
    i = tf.cast(i,tf.int64)
    return tf.data.Dataset.range(i)

ds_of_ds = (
    tf.data.Dataset.from_tensor_slices([i for i in range(1,11)])
    .map(create_ds)
    )

for ds in ds_of_ds:
    print(f"{type(ds)},length={len(ds)}")
    for item in ds:
        print(item.numpy(), end="")
    print()


If you apply flat_map instead of map, the elements of the inner Dataset are strung together to form a Dataset of integers.


In [ ]:
flat_ds = (tf.data.Dataset.from_tensor_slices(
    [i for i in range(1,11)])
    .flat_map(create_ds)
    )

# YOUR CODE HERE TO PRINT THE CONTENTS OF THIS DATASET
for flat in flat_ds:
    print(flat)


Finally, replace flat_map(create_ds) by interleave(create_ds, cycle_length=2) and observe the difference in output!

In [ ]:
flat_ds = (tf.data.Dataset.from_tensor_slices(
    [i for i in range(1,11)])
    .interleave(create_ds,cycle_length=2)
    )

# YOUR CODE HERE TO PRINT THE CONTENTS OF THIS DATASET
for flat in flat_ds:
    print(flat)



```text
Zakje1: [0]
Zakje2: [0, 1]
Zakje3: [0, 1, 2]

Flatmap:
Zakje1 -> 0
Zakje2 -> 0 1
Zakje3 -> 0 1 2
Interleave (cycle_length=3):
Zakje1 -> 0
Zakje2 -> 0
Zakje3 -> 0

Zakje1 -> 1
Zakje2 -> 1
Zakje3 -> 1
```

## 2  IMDB Movie Reviews (Bis)
In chapter 10, we already used thepreprocessedIMDB movie reviews dataset.In this exercise, we are going to use therawIMDB movie reviews dataset(which is more representative of real world problems), and build a binary classification model. We will use the keras preprocessing layers, as well as the tf.data.Dataset API.

This exercise is similar to, but not exactly the same as, exercise 10 in chapter 13 of the book.

### 2.1  Download the dataset
Download the dataset from the URL https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz using the keras.utils.get_file utility function. Use the appropriate arguments toget_fileso that:
- the downloaded file is unzipped and untarred
- the file is stored in the current directory (i.e. the directory where you are running the notebook)

Check that the file has been downloaded correctly by verifying that you have a directory called `datasets/aclImdb` in the current directory. This directory should contain the *train* and *test* subdirectories. Each of these directories in turn should contain a pos and neg subdirectory, which in turn contain the actualreviews. You can see the resulting directory structure in Figure1.

Note: there are other files and directories present, but we are not going to usethose. Also note that you can use!lsfrom within the notebook (if the underlying operating system understands thelscommand.)

In [ ]:
import os
import shutil
import random as rnd

URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
IMDB_ds = ks.utils.get_file(origin=URL,
                            fname="aclImdb.tar.gz",
                            extract=True,
                            cache_dir="datasets",
                            cache_subdir="")
if os.path.exists("datasets/aclImdb_extracted/aclImdb"):
    shutil.move("datasets/aclImdb_extracted/aclImdb","datasets/aclImdb")
    shutil.rmtree("datasets/aclImdb_extracted")

## 2.2  Create a Validation Directory
The dataset as downloaded contains 25000 train and 25000 test examples, with 12500 positive and negative reviews in each set. We are going to use all 25000 examples for training, but we are going to use 15000 of the examples in the test directory for validation. To be more specific, we are going to use 7500 positive and 7500 negative (test) reviews for validation.

Note: you could do this outside of Python, but we are going to use Python in this exercise, so that the notebook is fully self contained.

- Use `os.makedir` to create directories called val/neg and val/pos next to the existing train and test directories.
- Use `os.listdir` to get the list of files in the `test/neg` directory. Use `random.shuffle` to shuffle this list. Use `shutil.move` to move the first 7500 files to the val/neg directory.
- Do the same for the `test/pos` directory.

In [ ]:
VAL_PATH = "./datasets/aclImdb/validation"
TRAIN_PATH = "./datasets/aclImdb/train"
TEST_PATH = "./datasets/aclImdb/test"
POS_DIR = "/pos"
NEG_DIR = "/neg"

if not os.path.exists(VAL_PATH):
    os.mkdir(VAL_PATH)

test_files_pos = os.listdir(TEST_PATH+POS_DIR)
rnd.shuffle(test_files_pos)
test_files_pos = test_files_pos[:7500]

if not os.path.exists(VAL_PATH + POS_DIR):
    os.mkdir(VAL_PATH + POS_DIR)

for f_pos in test_files_pos:
    shutil.move(os.path.join(TEST_PATH+POS_DIR,f_pos), os.path.join(VAL_PATH + POS_DIR,f_pos))

test_files_neg = os.listdir(TEST_PATH+NEG_DIR)
rnd.shuffle(test_files_neg)
test_files_neg = test_files_neg[:7500]

if not os.path.exists(VAL_PATH + NEG_DIR):
    os.mkdir(VAL_PATH + NEG_DIR)

for f_neg in test_files_neg:
    shutil.move(os.path.join(TEST_PATH+NEG_DIR,f_neg), os.path.join(VAL_PATH+NEG_DIR,f_neg))

test_files_pos = os.listdir(TEST_PATH+POS_DIR)
print("test_files_pos len:",len(test_files_pos))

test_files_neg = os.listdir(TEST_PATH+NEG_DIR)
print("test_files_neg len:" ,len(test_files_neg))

val_files_pos = os.listdir(VAL_PATH+POS_DIR)
print("val_files_pos len:" ,len(val_files_pos))

val_files_neg = os.listdir(VAL_PATH+NEG_DIR)
print("val_files_neg len:" ,len(val_files_neg))


## 2.3  Create `tf.data.Dataset` objects
### 2.3.1  “Small Data” Option 
Use the fact that we still have “small” data, and use pure Python code to load all the reviews into main memory and then use `from_tensor_slices` to create the `tf.data.Dataset` objects

In [ ]:
def create_dataset_1(base_dir):
    # Approach 1:read all reviews into a list and use from_tensor_slices.
    # base_dir: directory name like "./datasets/aclImdb/train"
    reviews = []
    sentiments = []
    for sentiment in["pos","neg"]:
        directory = os.path.join(base_dir,sentiment)
        for file_path in os.listdir(directory):
            with open(os.path.join(directory,file_path),"r") as file:
                reviews.append(file.readline())
            sentiments.append(1.0 if sentiment == "pos" else 0.0)
    return tf.data.Dataset.from_tensor_slices((reviews,sentiments))

Use the following code to print three (positive) reviews from the test set

In [ ]:
for X, y in create_dataset_1("./datasets/aclImdb/test").take(3):
    print(X)
    print(y)
    print("*"*50)

You may get something like (although the actual text will be different):
```python
tf.Tensor([b'For comedy to work, there are many factors involved:<br /><br />1.tf.Tensor(1.0, shape=(), dtype=float32)**************************************************tf.Tensor([b"It\'s all about getting what you want when you want it. And the") tf.Tensor(1.0, shape=(), dtype=float32
```

2.3.2  Use `TextLineDataset`
Use `tf.data.TextLineDataset` to read the reviews directly from the files. This is a bit more complicated, but should be more scalable. You can use the following code to read the reviews from the files:


In [ ]:
def create_dataset_2(base_dir):
    pos_file_paths = [os.path.join(base_dir,"pos", f)
                      for f in os.listdir(os.path.join(base_dir,"pos"))]

    neg_file_paths = [os.path.join(base_dir,"neg", f)
                      for f in os.listdir(os.path.join(base_dir,"neg"))]
    pos_ds = tf.data.TextLineDataset(pos_file_paths,num_parallel_reads=4).map(lambda review: (review, 1.0))
    neg_ds = tf.data.TextLineDataset(neg_file_paths,num_parallel_reads=4).map(lambda review: (review, 0.0))
    return pos_ds.concatenate(neg_ds)

You will have to think of a way to:
- Attach the labels to the reviews.
- Combine the twotf.data.Datasets into a single `tf.data.Dataset`.

The code above to print some reviews should still work if you replace `create_dataset_1` by `create_dataset_2`

### 2.3.3  Create the Training, Validation and Test sets
Finally, create the actual dataset objects that will be used to train the model. 
- We need to shuffle the training data (but not the validation nor the test data). 
- All datasets need to be batched. Use a batch size of 512 (but make this easily changeable). Use prefetching on all datasets

`Prefetch` zorgt voor overlap tussen data-voorbereiding en training.
- Zonder: De computer leest data $\rightarrow$ wacht $\rightarrow$ traint $\rightarrow$ wacht $\rightarrow$ leest data. (CPU en GPU wachten op elkaar).
- Met: Terwijl de GPU traint op Batch 1, bereidt de CPU op de achtergrond Batch 2 alvast voor


In [154]:
BATCH_SIZE = 512
train_ds = (
    create_dataset_2(TRAIN_PATH)
    .shuffle(buffer_size=15_000,seed=42)
    .batch(BATCH_SIZE)
    .prefetch(1)
)
val_ds = (
    create_dataset_2(VAL_PATH)
    .batch(BATCH_SIZE)
    .prefetch(1)
)
test_ds = (
    create_dataset_2(TEST_PATH)
    .batch(BATCH_SIZE)
    .prefetch(1)
)

## 2.4  Create and train a Model with a Multi-Hot Encoding
In the exercise in chapter 10, we encoded each review as a multi-hot encoded sequence, and we “hand-coded” this conversion logic. Use the `TextVectorization` layer to convert the reviews to multi-hot encoded sequences. The vocabulary should be limited to 10000 tokens.

Call your layer `multi_hot_layer`. Next, create a function called `get_model` that is basically the same as the `get_model` function from the exercise in chapter 10, but takes an argument called `conversion_layer`. This argument is inserted as the first actual layer in the model (after the input layer which specifiesthe shape and the data-type of the input).


In [ ]:
for text_batch, label_batch in train_ds.take(1):
    print("Shape van de tekst-batch:", text_batch.shape)
    print("Shape van de label-batch:", label_batch.shape)
print(train_ds.counter())

In [ ]:
vectorize_layer = ks.layers.TextVectorization(
    max_tokens=10_000,
    output_mode="multi_hot",
    name="mutli_hot_layer"
)
vectorize_layer.adapt(train_ds.map(lambda x, y: x))


def get_model(conversion_layer):
    return ks.Sequential([
        ks.Input(shape=(), dtype=tf.string),
        conversion_layer,

        ks.layers.Dense(16, activation="relu"),
        ks.layers.Dense(16, activation="relu"),

        ks.layers.Dense(1,activation="sigmoid")
    ])


model = get_model(vectorize_layer)
model.compile(
    loss="binary_crossentropy",
    metrics=["accuracy"])

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[ks.callbacks.EarlyStopping(patience=2)]
)

Next, compile and train the model as in chapter 10. Use anEarlyStoppingcallback to prevent (too much) overfitting.Evaluate your model on the validation set

In [ ]:
model.evaluate(val_ds)

## 2.5  Create and train a Model with a TF-IDF Encoding
Repeat the previous step, but now use the TextVectorization layer to convert the reviews to TF-IDF encoded sequences. Check that this second model has
the same number of parameters as the first one, since we are still using a sparse representation of the reviews.

You should find that the model with TF-IDF performs very similar to (butslightly worse than) the model with multi-hot encoding.

In [159]:
vectorize_layer = ks.layers.TextVectorization(
    max_tokens=10_000,
    output_mode="tf_idf",
    name="mutli_hot_layer",
)

with tf.device('/CPU:0'):
    vectorize_layer.adapt(train_ds.map(lambda x, y: x))

tf_idf_model = get_model(vectorize_layer)
tf_idf_model.compile(
    loss="binary_crossentropy",
    metrics=["accuracy"])

tf_idf_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[ks.callbacks.EarlyStopping(patience=2)]
)

Epoch 1/10


2025-12-29 17:16:16.172110: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


     49/Unknown 3s 27ms/step - accuracy: 0.7370 - loss: 0.5981

2025-12-29 17:16:18.795720: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-12-29 17:16:18.795736: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9748362425322897822
2025-12-29 17:16:18.795741: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/Cast_1/_3]]
2025-12-29 17:16:18.795749: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 1675191259102476621
2025-12-29 17:16:18.795752: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16694153430287954310
2025-12-29 17:16:18.795778: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 877596677921548

49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.7812 - loss: 0.5075 - val_accuracy: 0.8317 - val_loss: 0.4190
Epoch 2/10


...

2025-12-29 17:16:26.593833: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-12-29 17:16:26.593847: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2665769967199814456
2025-12-29 17:16:26.593851: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9735635423891166250
2025-12-29 17:16:26.593853: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/sequential_13_1/mutli_hot_layer_1/StringSplit/strided_slice_1/_12]]
2025-12-29 17:16:26.593859: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8402937684174718300
2025-12-29 17:16:26.593863: I tensorflow/core/framework/local_rendezvous.cc:422] Local re

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9550 - loss: 0.1310

2025-12-29 17:16:28.206026: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-12-29 17:16:28.206044: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9748362425322897822
2025-12-29 17:16:28.206047: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/sequential_13_1/mutli_hot_layer_1/SelectV2/_22]]
2025-12-29 17:16:28.206051: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13115765615121797607
2025-12-29 17:16:28.206054: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8775966779215483281
2025-12-29 17:16:28.206059: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item

49/49 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9524 - loss: 0.1362 - val_accuracy: 0.8705 - val_loss: 0.3724


2025-12-29 17:16:28.928067: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-12-29 17:16:28.928086: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2665769967199814456
2025-12-29 17:16:28.928093: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9735635423891166250
2025-12-29 17:16:28.928096: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8402937684174718300
2025-12-29 17:16:28.928099: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12031178677076522404
2025-12-29 17:16:28.928102: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17801456960509093524
2025-12-29 17:16:28.928104: I tensorflow/core/framework/local_rendezv